# Scikit-Learn 문서 특징 추출 기능

- DictVectorizer: 각 단어의 수를 세어놓은 사전에서 BoW(Bag-of-Words) 인코딩 벡터를 만든다.

- CountVectorizer: 문서 집합에서 단어 토큰을 생성하고 각 단어의 수를 세어 BoW 인코딩 벡터를 만든다.

- TfidfVectorizer: CountVectorizer와 비슷하지만 TF-IDF 방식으로 단어의 가중치를 조정한 BoW 인코딩 벡터를 만든다.

- HashingVectorizer: 해시 함수(hash function)을 사용하여 적은 메모리와 빠른 속도로 BoW 인코딩 벡터를 만든다.



## DictVectorizer
- 단어의 카운트(출현 빈도, frequency) 나타내는 Dictionary 정보를 입력받아 벡터화

In [11]:
from sklearn.feature_extraction import DictVectorizer

v = DictVectorizer(sparse=False)
D = [{'foo': 1, 'bar': 2}, {'foo': 3, 'baz': 1}]
X = v.fit_transform(D)

In [12]:
print(X)

[[2. 0. 1.]
 [0. 1. 3.]]


In [13]:
print(v.feature_names_)

['bar', 'baz', 'foo']


In [14]:
v.inverse_transform(X)

[{'bar': 2.0, 'foo': 1.0}, {'baz': 1.0, 'foo': 3.0}]

In [15]:
v.transform({'foo': 4, 'unseen_feature': 3})

array([[0., 0., 4.]])

## CountVectorizer
- 단어들의 카운트(출현 빈도, frequency)로 여러 문서들을 벡터화


In [16]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = [
    'This is the first document.',
    'This is the second second document.',
    'And the third one.',
    'Is this the first document?',
    'The last document?',
]
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)

In [17]:
print(vectorizer.get_feature_names())

['and', 'document', 'first', 'is', 'last', 'one', 'second', 'the', 'third', 'this']


In [18]:
print(X.toarray())

[[0 1 1 1 0 0 0 1 0 1]
 [0 1 0 1 0 0 2 1 0 1]
 [1 0 0 0 0 1 0 1 1 0]
 [0 1 1 1 0 0 0 1 0 1]
 [0 1 0 0 1 0 0 1 0 0]]


In [19]:
print(X.toarray())

[[0 1 1 1 0 0 0 1 0 1]
 [0 1 0 1 0 0 2 1 0 1]
 [1 0 0 0 0 1 0 1 1 0]
 [0 1 1 1 0 0 0 1 0 1]
 [0 1 0 0 1 0 0 1 0 0]]


### Stop Words
- Stop Words 는 문서에서 단어장을 생성할 때 무시할 수 있는 단어를 말한다. 
- 보통 영어의 관사나 접속사, 한국어의 조사 등이 여기에 해당한다.

In [20]:
vectorizer = CountVectorizer(stop_words=["and", "is", "the", "this"]).fit(corpus)
print(vectorizer.vocabulary_)

{'first': 1, 'document': 0, 'second': 4, 'third': 5, 'one': 3, 'last': 2}


In [21]:
vectorizer = CountVectorizer(stop_words="english").fit(corpus)
print(vectorizer.vocabulary_)

{'document': 0, 'second': 1}


### 토큰
- analyzer, tokenizer, token_pattern 등의 인수로 사용할 토큰 생성기를 선택할 수 있다.


In [22]:
vectorizer = CountVectorizer(analyzer="char").fit(corpus)
print(vectorizer.vocabulary_)

{'t': 16, 'h': 8, 'i': 9, 's': 15, ' ': 0, 'e': 6, 'f': 7, 'r': 14, 'd': 5, 'o': 13, 'c': 4, 'u': 17, 'm': 11, 'n': 12, '.': 1, 'a': 3, '?': 2, 'l': 10}


In [23]:
vectorizer = CountVectorizer(token_pattern="t\w+").fit(corpus)
print(vectorizer.vocabulary_)

{'this': 2, 'the': 0, 'third': 1}


In [24]:
import nltk

nltk.download('punkt')

vectorizer = CountVectorizer(tokenizer=nltk.word_tokenize).fit(corpus)
print(vectorizer.vocabulary_)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
{'this': 11, 'is': 5, 'the': 9, 'first': 4, 'document': 3, '.': 0, 'second': 8, 'and': 2, 'third': 10, 'one': 7, '?': 1, 'last': 6}


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:507: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn("The parameter 'token_pattern' will not be used"


### N그램
- N그램은 단어장 생성에 사용할 토큰의 크기를 결정한다. 
- 모노그램(monogram)은 토큰 하나만 단어로 사용한다. 
- 바이그램(bigram)은 두 개의 연결된 토큰을 하나의 단어로 사용한다.

In [25]:
vectorizer2 = CountVectorizer(analyzer='word', ngram_range=(2, 2))
X2 = vectorizer2.fit_transform(corpus)
print(vectorizer2.get_feature_names())

['and the', 'first document', 'is the', 'is this', 'last document', 'second document', 'second second', 'the first', 'the last', 'the second', 'the third', 'third one', 'this is', 'this the']


### 빈도수
- max_df, min_df 인수를 사용하여 문서에서 토큰이 나타난 횟수를 기준으로 단어장을 구성할 수도 있다. 
- 토큰의 빈도가 max_df로 지정한 값을 초과 하거나 min_df로 지정한 값보다 작은 경우에는 무시한다. 
- 인수 값은 정수인 경우 횟수, 부동소수점인 경우 비중을 뜻한다.

In [26]:
vectorizer = CountVectorizer(max_df=4, min_df=2).fit(corpus)
print(vectorizer.vocabulary_)
print(vectorizer.stop_words_)

{'this': 3, 'is': 2, 'first': 1, 'document': 0}
{'last', 'third', 'second', 'and', 'the', 'one'}


In [29]:
vectorizer.transform(corpus).toarray().sum(axis=0)

array([4, 2, 3, 3])

## TF-IDF(Term Frequency – Inverse Document Frequency)
- 단어의 빈도와 역 문서 빈도를 사용하여 DTM 내의 각 단어들마다 중요한 정도를 가중치로 주는 방법
- 문서의 유사도를 구하는 작업, 검색 시스템에서 검색 결과의 중요도를 정하는 작업, 문서 내에서 특정 단어의 중요도를 구하는 작업 등에 사용

In [30]:
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = [
    'This is the first document.',
    'This is the second second document.',
    'And the third one.',
    'Is this the first document?'
]

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)

print(vectorizer.get_feature_names())

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']


In [31]:
print(X.shape)

(4, 9)


In [32]:
print(X.toarray())

[[0.         0.43877674 0.54197657 0.43877674 0.         0.
  0.35872874 0.         0.43877674]
 [0.         0.27230147 0.         0.27230147 0.         0.85322574
  0.22262429 0.         0.27230147]
 [0.55280532 0.         0.         0.         0.55280532 0.
  0.28847675 0.55280532 0.        ]
 [0.         0.43877674 0.54197657 0.43877674 0.         0.
  0.35872874 0.         0.43877674]]


## HashingVectorizer
- 해시 함수(hash function)을 사용하여 적은 메모리와 빠른 속도로 BoW 인코딩 벡터를만든다.

In [ ]:
from sklearn.feature_extraction.text import HashingVectorizer

corpus = [
    'This is the first document.',
    'This is the second second document.',
    'And the third one.',
    'Is this the first document?'
]

vectorizer = HashingVectorizer(n_features=2**4)
X = vectorizer.fit_transform(corpus)

In [34]:
print(X.shape)

(4, 16)


In [35]:
print(X.toarray())

[[-0.57735027  0.          0.          0.          0.          0.
   0.          0.         -0.57735027  0.          0.          0.
   0.          0.57735027  0.          0.        ]
 [-0.40824829  0.          0.          0.          0.          0.
   0.          0.          0.          0.          0.          0.81649658
   0.          0.40824829  0.          0.        ]
 [ 0.          0.          0.          0.         -0.5         0.5
   0.          0.          0.          0.          0.          0.
   0.         -0.5        -0.5         0.        ]
 [-0.57735027  0.          0.          0.          0.          0.
   0.          0.         -0.57735027  0.          0.          0.
   0.          0.57735027  0.          0.        ]]
